# IE4424 Lab Time_Series_Prediction

## Acknowledgment

This lab experiment is inspired by Usman Malik's Stack Abuse article.

You can check out the original post at https://stackabuse.com/time-series-prediction-using-lstm-with-pytorch-in-python



## 1. Training a time series predictor

A time series data is a sequential data indexed in time order.
Often we would like to predict the future behaviour of a time series.
For example, we may be interested in predicting the weather, share prices, sales, etc. 
Long Short-Term Memory (LSTM) model can be applied to perform time series prediction. 

In this exercise, we will use the ``flights`` dataset from the seaborn library.
The dataset contains 3 columns: ``year``, ``month``, and ``passengers``.
Where the ``year`` and ``month`` columns refer to the particular year and month of the monthly flight record, respectively.
The ``passengers`` column records the total number of passengers that took the flights in that month.

We will do the following steps in order:

1) Load and normalize the training and testing datasets using ``StandardScalar``
   
2) Define an LSTM model

3) Define a loss function

4) Train the network on the training data

5) Test the network on the test data

In [ ]:
%matplotlib inline

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(0)
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

### 1.1 Loading the flights dataset


Loading the ``flights`` data from ``seaborn`` is strightforward.

In [ ]:
import seaborn as sns

flights = sns.load_dataset('flights')
print(type(flights)) # the dataset is loaded as pandas dataframe
flights.describe(include = 'all')

# save the data as comma-separated values (csv) to facilitate viewing of the raw data
flights.to_csv('flights_data.csv', index=False)


### 1.2 Visualizing the training data


In [ ]:
plt.plot(flights['passengers'], 'k')
plt.title('flight records')
plt.xlabel('Months')
plt.ylabel('Passengers')
plt.show()

### 1.3 Normalizing the data and converting to tensor

In a regression problem such as this, it is beneficial to normalize the data. It allows the model to converge faster and avoid having large losses. 

In [ ]:
from sklearn.preprocessing import StandardScaler

passengers = flights['passengers'].values.astype(float) # convert the passengers column to float 
scaler = StandardScaler()
normalized_data = scaler.fit_transform(passengers.reshape(-1,1))

### 1.4 Generating the sequences
Since we have 144 months (i.e., 12 years) of flight data, we use 132 months (i.e., 11 years) for training and the remaining 12 months (i.e., 1 year) for testing.

In [ ]:
# create windowed sequence tuples of (x,y), with moving window of step size = 1
def windowed_sequences(data, window_size=12, test_size=12):
    x = []
    y = []
    for i in range(len(data)-window_size-test_size+1):
        j = i + window_size
        xi = data[i:j]
        yi = data[j]
        x.append(xi)
        y.append(yi)
    return x, y
        
test_size = 12
seqeunces, targets = windowed_sequences(normalized_data, 12, test_size)

 # Convert the sequences to float type tensors
train_x = torch.Tensor(seqeunces[:-1]).type(torch.float)
train_y = torch.Tensor(targets[:-1]).type(torch.float) 
test_seq = torch.Tensor(seqeunces[-1]).type(torch.float)

### 1.5 Constructing the iterables

In [ ]:
# Using pytorch data loading utility to construct iterable over the dataset

from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, x, y):
        super(CustomDataset, self).__init__()
        self.x = x
        self.y = y

    def __getitem__(self, item):
        return self.x[item],  self.y[item]

    def __len__(self):
        return len(self.x)
    
train_set = CustomDataset(train_x, train_y)
train_loader = DataLoader(train_set, batch_size=24)

### 1.6 Defining the LSTM model

Since we only have 1 input and 1 output value which is the number of passengers, we set the input dimension (input_dim) and output dimension (output_dim) as 1.
hidden_dim defines the number of features in the hidden state of the LSTM.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class TimeSeriesPredictor(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=200, output_dim=1):
        super(TimeSeriesPredictor,self).__init__()
        
        self.hidden_dim = hidden_dim
        
        self.lstm = nn.LSTM(input_dim, self.hidden_dim,
                            batch_first=True)
                
        self.fc1 = nn.Linear(self.hidden_dim, 100)
        self.fc2 = nn.Linear(100, output_dim)

    def forward(self, input_seq):

        lstm_out, (hn, cn) = self.lstm(input_seq)
        predictions = F.relu(self.fc1(hn.view(-1, self.hidden_dim)))
        predictions = self.fc2(predictions)
        return predictions
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  #2024 modification CPU > GPU
model = TimeSeriesPredictor()
model.to(device)#2024 modification CPU > GPU

### 1.7 Printing the network structure

In [ ]:
print(model)

### 1.8 Using Torchinfo to view a summary of the network parameters

In [ ]:
!pip install torchinfo #2024 modification CPU > GPU
from torchinfo import summary
summary(model, input_size=(24,12,1)) # (batch_size, sequence_length, input_dim)

### 1.9 Defining loss function and optimizer

The loss function is defined as Mean Squared Error loss and optimizer is Adam optimizer. 



In [ ]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

### 1.10 Training the network

We loop over our data iterator, and feed the inputs to the
network and optimize the parameters.



In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Print the device being used
print(f"Using device: {device}")


model.to('cpu')
t1 = time.time()
loss_list = []
for epoch in range(1000):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        seq, labels = data
        optimizer.zero_grad()
        y_pred = model(seq)
        loss = criterion(y_pred, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
    loss_list.append(running_loss/len(train_loader))
    if epoch % 100 == 99:
        print(f'epoch: {epoch+1:3} loss: {running_loss/len(train_loader):.4f}')

t2 = time.time()
print('Finished Training')
print('Training time:'+str(t2-t1))

## Exercise 1.1 Plotting of loss function

In the previous cell, we have saved the loss function at different iterations in the loss_list.

Now, plot the loss function versus the iteration number using matplotlib.

You can find the tutorial of plotting figures using matplotlib here:
https://matplotlib.org/3.3.3/gallery/lines_bars_and_markers/simple_plot.html#sphx-glr-gallery-lines-bars-and-markers-simple-plot-py

### E1.1.1 Example of using matplotlib.pyplot

In [ ]:
# An example of a simple plot
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

# Data for plotting
t = np.arange(0.0, 2.0, 0.01)
s = 1 + np.sin(2 * np.pi * t)

fig, ax = plt.subplots()
ax.plot(t, s)

ax.set(xlabel='time (s)', ylabel='voltage (mV)',
       title='About as simple as it gets, folks')
ax.grid()

plt.show()

### E1.1.2 Plotting the loss function vs interation number (To do)

In [ ]:
# Plot the training loss curve
# To do


### 1.11 Evaluating network performance on the test data

We have trained the network for several passes over the training dataset.
But we need to check if the network has learned anything at all.

We will check this by predicting the next time-step value that the neural network
outputs, and checking it against the ground truth.



In [ ]:
x_axis = np.arange(132, 144, 1)
plt.title('Month vs Passenger')
plt.ylabel('Passengers')
plt.grid(True)
plt.autoscale(axis='x', tight=True)
plt.plot(x_axis, passengers[-test_size:])
plt.show()

### 1.12 Perform predictions using the trained model



In [ ]:
preds_sl12 = []
with torch.no_grad():
    for i in range(test_size):
        outputs = model(test_seq.unsqueeze(dim=0)) # unsqueeze to add a dimension to accomodate the batch processing
        test_seq = torch.cat((test_seq,outputs),dim=0)[-12:]
        preds_sl12.append(outputs)

## Exercise 1.2 Evaluating network performance 

In this section, we are going to evaluate the accuracy of the model in predicting future values. Since we are predicting more than 1 value, we will use the Mean Absolute Percentage Error (MAPE) to measure the difference between the predicted value and the ground truth value. 

### E1.2.1 Computing MAPE (To do)

Finish the code and calculate the MAPE for 12 future predictions. They can be computed using API from scikit-learn:

MAPE: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_percentage_error.html

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error

denormalized_preds_sl12 = scaler.inverse_transform(np.array(preds_sl12).reshape(-1, 1))

# To do
mape = 

print('MAPE: %.2f%%' % (mape*100))

### E1.2.2 Plotting the predictions against the data

In [ ]:
x_axis = np.arange(132, 144, 1)
plt.title('Month vs Passenger')
plt.ylabel('Passengers')
plt.grid(True)
plt.autoscale(axis='x', tight=True)
plt.plot(passengers, 'k')
plt.plot(x_axis,denormalized_preds_sl12, 'r^--', label='seq_len_12')
plt.axvline(x=132, color='r', linestyle='--')
plt.show()

### E1.2.3 Zoom in to the predictions

In [ ]:
x_axis = np.arange(132, 144, 1)
plt.title('Month vs Passenger')
plt.ylabel('Passengers')
plt.grid(True)
plt.autoscale(axis='x', tight=True)
plt.plot(x_axis,passengers[-test_size:], 'ko-', label='ground_truth')
plt.plot(x_axis,denormalized_preds_sl12, 'r^--', label='seq_len_12')
plt.legend()
plt.show()

## E1.3 Study of different window/sequence lengths

In this exercise, you will learn to train the network with the input of a different window/sequence length.

In the previous section, we used a sequence length of 12 for the time series input. You will use a sequence length of 24 in this exercise and compare the evaluation results for these two window/sequence lengths.

### E1.3.1 Prepare the new training iterables (To do)

In [ ]:
# To do
seqeunces, targets =
train_x = torch.Tensor(seqeunces[:-1]).type(torch.float)
train_y = torch.Tensor(targets[:-1]).type(torch.float)
test_seq = torch.Tensor(seqeunces[-1]).type(torch.float)

train_set = CustomDataset(train_x, train_y)
train_loader = DataLoader(train_set, batch_size=24)
test_seq = torch.Tensor(seqeunces[-1]).type(torch.float)

### E1.3.2 Re-initializing the network and optimizer

In [ ]:
model = TimeSeriesPredictor()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

### E1.3.3 Training the network with the new input sequences

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")#2024 modification CPU > GPU
model.to(device)#2024 modification CPU > GPU
t1 = time.time()
loss_list = []
for epoch in range(1000):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        seq, labels = data
        seq, labels = seq.to(device), labels.to(device)#2024 modification CPU > GPU
        optimizer.zero_grad()
        y_pred = model(seq)
        loss = criterion(y_pred, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    loss_list.append(running_loss/len(train_loader))
    if epoch % 100 == 99:
        print(f'epoch: {epoch+1:3} loss: {running_loss/len(train_loader):.4f}')

t2 = time.time()
print('Finished Training')
print('Training time:'+str(t2-t1))

### E1.3.4 Perform predictions

Perform predictions using the model trained on inputs sequence length = 24.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")#2024 modification CPU > GPU
model.to(device)#2024 modification CPU > GPU
preds_sl24 = []
with torch.no_grad():
    for i in range(test_size):
        outputs = model(test_seq.unsqueeze(dim=0)) # unsqueeze to add a dimension to accomodate the batch processing
        test_seq = torch.cat((test_seq,outputs),dim=0)[-24:]
        preds_sl24.append(outputs)

### E1.3.5 Evaluate performance

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
import numpy as np

# Check if preds_sl24 is a list, and convert to a NumPy array
if isinstance(preds_sl24, list):
    # If it's a list of tensors, we need to convert each tensor to NumPy
    preds_sl24 = np.concatenate([x.cpu().detach().numpy().flatten() if isinstance(x, torch.Tensor) else np.array(x).flatten() for x in preds_sl24])#2024 modification CPU > GPU

# Now preds_sl24 is a 1D NumPy array, reshape it to (n_samples, 1) for the scaler
denormalized_preds_sl24 = scaler.inverse_transform(preds_sl24.reshape(-1, 1))#2024 modification CPU > GPU

# Calculate MAPE
mape = mean_absolute_percentage_error(passengers[-test_size:], denormalized_preds_sl24)

# Print the MAPE as percentage
print(f'MAPE: {mape * 100:.2f}%')


### E1.3.6 Plotting the results for both sequence lengths (To do)


Re-plot the plot in E1.2.3 by adding the input-sequence-length-24 prediction. 

Specifically, plot the ground truth (in black and label it as 'ground truth'), the sequence-length-12 prediction (in red dotted line with ^ marker and label it as 'seq_len_12'), and the sequence-length-24 prediction (in green dotted line with x marker and label it as 'seq_len_24').

In [ ]:
# To do


## Exercise 1.4 Study of different hidden dimensions

In this exercise, you will learn to train the network using a different hidden layer dimension.

In the previous section, we used a hidden dimension of 200. You will use a hidden dimension of 400 in this exercise and compare the results for these 2 hidden dimensions. We will keep the window/sequence length of 12.

### E1.4.1 Re-initializing the network with different hidden dimensions (To do)

In [ ]:
# To do
model = 
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001) # lower learning may be better for more complex model

### E1.4.2 View a summary of the network parameters

In [ ]:
from torchinfo import summary
summary(model, input_size=(24,12,1))

### E1.4.3 Reinitialize the training iterables

In [ ]:
window_size = 12
test_size = 12
seqeunces, targets = windowed_sequences(normalized_data, window_size, test_size)
train_x = torch.Tensor(seqeunces[:-1]).type(torch.float)
train_y = torch.Tensor(targets[:-1]).type(torch.float)
test_seq = torch.Tensor(seqeunces[-1]).type(torch.float)

train_set = CustomDataset(train_x, train_y)
train_loader = DataLoader(train_set, batch_size=24)
test_seq = torch.Tensor(seqeunces[-1]).type(torch.float)

### E1.4.4 Training the network with new hidden dimension

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)#2024 modification CPU > GPU
t1 = time.time()
loss_list = []
for epoch in range(1000):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        seq, labels = data
        seq, labels = seq.to(device), labels.to(device)#2024 modification CPU > GPU
        optimizer.zero_grad()
        y_pred = model(seq)
        loss = criterion(y_pred, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    loss_list.append(running_loss/len(train_loader))
    if epoch % 100 == 99:
        print(f'epoch: {epoch+1:3} loss: {running_loss/len(train_loader):.4f}')

t2 = time.time()
print('Finished Training')
print('Training time:'+str(t2-t1))

### E1.4.5 Perform predictions

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")#2024 modification CPU > GPU
model.to(device)#2024 modification CPU > GPU
preds_hd400 = []
with torch.no_grad():
    for i in range(test_size):
        outputs = model(test_seq.unsqueeze(dim=0)) # unsqueeze to add a dimension to accomodate the batch processing
        test_seq = torch.cat((test_seq,outputs),dim=0)[-12:]
        preds_hd400.append(outputs)

### E1.4.6 Evaluate performance

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
import numpy as np

# If preds_hd400 is a list, convert it to a NumPy array
if isinstance(preds_hd400, list):
    # Convert the list into a 1D NumPy array
    preds_hd400 = np.concatenate([x.cpu().detach().numpy().flatten() if isinstance(x, torch.Tensor) else np.array(x).flatten() for x in preds_hd400])#2024 modification CPU > GPU

# If preds_hd400 is a tensor (after conversion from the list), move it to CPU
elif isinstance(preds_hd400, torch.Tensor):
    preds_hd400 = preds_hd400.cpu().detach().numpy() #2024 modification CPU > GPU

# Reshape the array to a column vector (if needed) before inverse transform
denormalized_preds_hd400 = scaler.inverse_transform(preds_hd400.reshape(-1, 1))#2024 modification CPU > GPU

# Calculate MAPE
mape = mean_absolute_percentage_error(passengers[-test_size:], denormalized_preds_hd400)

# Print the MAPE as percentage
print(f'MAPE: {mape * 100:.2f}%')

### E1.4.7 Plotting the results in the same plot (To do)

Re-plot the plot in E1.2.3 by adding the hidden-dimension-400 prediction. 

Specifically, plot the the ground truth (in black and label it as 'ground truth'), the hidden-dimension-200 prediction (in red dotted line with ^ marker and label it as 'hid_dim_200'), and the hidden-dimension-400 prediction (in green dotted line with x marker and label it as 'hid_dim_400').

In [ ]:
# To do 
